In [1]:
import pandas as pd

## 1. 고객 기본정보 DB 넣기

In [2]:
excel_file ='./files/01.고객 기본정보 DB (샘플).xlsx'
df = pd.read_excel(excel_file)
df.head()

,userID,성함,주소,이메일,연락처,아이디,비밀번호,생년월일(6자리),성별(남/여)
0,M0001,공유,NaN,NaN,NaN,NaN,NaN,730905,남
1,F0002,송혜교,NaN,NaN,NaN,NaN,NaN,800520,여
2,F0003,이소연,NaN,NaN,NaN,NaN,NaN,910909,여
3,F0004,하양이,NaN,NaN,NaN,NaN,NaN,1111,여
4,F0005,괄괄이,NaN,NaN,NaN,NaN,NaN,20201,여


In [3]:
# 생년월일 0 패딩처리
df['생년월일(6자리)'] = df['생년월일(6자리)'].map('{:06d}'.format)


In [4]:
df.head()

,userID,성함,주소,이메일,연락처,아이디,비밀번호,생년월일(6자리),성별(남/여)
0,M0001,공유,NaN,NaN,NaN,NaN,NaN,730905,남
1,F0002,송혜교,NaN,NaN,NaN,NaN,NaN,800520,여
2,F0003,이소연,NaN,NaN,NaN,NaN,NaN,910909,여
3,F0004,하양이,NaN,NaN,NaN,NaN,NaN,001111,여
4,F0005,괄괄이,NaN,NaN,NaN,NaN,NaN,020201,여


In [5]:
df.columns

Index(['userID', '성함', '주소', '이메일', '연락처', '아이디', '비밀번호', '생년월일(6자리)',
       '성별(남/여)'],
      dtype='object')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   userID     9999 non-null   object 
 1   성함         9999 non-null   object 
 2   주소         1 non-null      object 
 3   이메일        0 non-null      float64
 4   연락처        0 non-null      float64
 5   아이디        0 non-null      float64
 6   비밀번호       0 non-null      float64
 7   생년월일(6자리)  9999 non-null   object 
 8   성별(남/여)    9999 non-null   object 
dtypes: float64(4), object(5)
memory usage: 703.2+ KB


In [7]:
# 열 이름 적절하게 변경
df.rename(columns={'성함':'name', '주소':'address', '이메일':'email', '연락처':'phonenum','아이디':'nickname','비밀번호':'password', '생년월일(6자리)':'birth','성별(남/여)':'sex'},inplace=True)

In [8]:
df.head()

,userID,name,address,email,phonenum,nickname,password,birth,sex
0,M0001,공유,NaN,NaN,NaN,NaN,NaN,730905,남
1,F0002,송혜교,NaN,NaN,NaN,NaN,NaN,800520,여
2,F0003,이소연,NaN,NaN,NaN,NaN,NaN,910909,여
3,F0004,하양이,NaN,NaN,NaN,NaN,NaN,001111,여
4,F0005,괄괄이,NaN,NaN,NaN,NaN,NaN,020201,여


In [9]:
# NaN값 '-'로 대체
df.fillna('-',inplace=True)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   userID    9999 non-null   object
 1   name      9999 non-null   object
 2   address   9999 non-null   object
 3   email     9999 non-null   object
 4   phonenum  9999 non-null   object
 5   nickname  9999 non-null   object
 6   password  9999 non-null   object
 7   birth     9999 non-null   object
 8   sex       9999 non-null   object
dtypes: object(9)
memory usage: 703.2+ KB


In [12]:
## DB연결후, 저장

import pymysql

conn = pymysql.connect(host='localhost', user='root', password='apple123!!',\
    db = 'beaulol', charset='utf8')
cur= conn.cursor()

cur.execute(""" 
            create table if not exists customer_info ( 
            userID VARCHAR(10) NOT NULL,
            name VARCHAR(30) NOT NULL, 
            address VARCHAR(200), 
            email VARCHAR(50), 
            phonenum  VARCHAR(30),
            nickname VARCHAR(20), 
            password VARCHAR(40), 
            birth VARCHAR(6) NOT NULL,
            sex VARCHAR(1) NOT NULL,
            PRIMARY KEY(userID)
            )
            """)

for idx,row in df.iterrows():
    try:
        sql = "INSERT INTO customer_info(userID,name,address,email,phonenum,nickname,password,birth,\
        sex) values(%s, %s, %s, %s, %s, %s, %s, %s, %s)"
        
        cur.execute(sql, (row['userID'],row['name'],row['address'],row['email'], row['phonenum'],\
            row['nickname'], row['password'],row['birth'],row['sex']))
        
    except Exception as e:
        print("Error:", e)
        continue

conn.commit()
conn.close()

## 2. 고객 성향정보 DB넣기

In [13]:
excel_file2 ='./files/02.고객 성향 DB (샘플).xlsx'
df2 = pd.read_excel(excel_file2)
df2.head()

,userID,피부톤,피부타입,피부고민,선호 스타일
0,NaN,쿨톤,지성,잡티(-미백-홍조-다크서클),청순
1,NaN,웜톤,건성,주름(-탄력),데일리
2,NaN,봄웜톤,복합성,각질,유니크(힙)
3,NaN,여름쿨톤,민감성,트러블,섹시
4,NaN,가을웜톤,중성,블랙헤드(-모공),러블리


In [14]:
len(df2['userID'])

10005

In [15]:
# 위에 설명행 제거
df2.drop(range(6),inplace=True)

In [16]:
len(df2['userID'])

9999

In [17]:
# # userID 데이터 값 추가
# df2['userID'] = ['M0001', 'F0002', 'F0003', 'F0004', 'F0005', 'F0006', 'M0007', 'F0008', 'F0009', 'F0010',
#                  'F0011', 'F0012', 'F0013', 'F0014', 'F0015', 'M0016', 'M0017', 'M0018', 'M0019', 'M0020',
#                  'F0021', 'F0022', 'F0023', 'F0024', 'F0025', 'F0026', 'F0027', 'F0028', 'F0029', 'F0030']

In [18]:
# 기초그룹의 NaN값 '-'로 변경
# df2['기초 그룹'].fillna('-',inplace=True)

In [19]:
# # 필요없는 열 제거
# df2.drop(columns=['Unnamed: 6','Unnamed: 7'],inplace=True)
# 열 이름에서 공백 제거
df2.rename(columns={'피부톤':'user_tone', '피부타입':'user_skintype', '피부고민':'user_trouble', '선호 스타일':'user_style'},inplace=True)

In [20]:
# 피부고민 열에서 하이픈'-'을 슬래시'/'로 변경
df2['user_trouble'] = df2['user_trouble'].str.replace('-','/')

# 괄호 제거
df2['user_trouble'] = df2['user_trouble'].str.replace('(','').str.replace(')','')

In [21]:
df2.head()

,userID,user_tone,user_skintype,user_trouble,user_style
6,M0001,쿨톤,복합성,블랙헤드/모공,유니크(힙)
7,F0002,가을웜톤,건성,민감성/아토피,데일리
8,F0003,겨울쿨톤,복합성,주름/탄력,유니크(힙)
9,F0004,봄웜톤,중성,잡티/미백/홍조/다크서클,러블리
10,F0005,겨울쿨톤,건성,각질,유니크(힙)


In [22]:
df2.columns

Index(['userID', 'user_tone', 'user_skintype', 'user_trouble', 'user_style'], dtype='object')

In [23]:
## DB연결후, 저장

import pymysql

conn = pymysql.connect(host='localhost', user='root', password='apple123!!',\
    db = 'beaulol', charset='utf8')
cur= conn.cursor()

cur.execute(""" 
            create table if not exists customer_type ( 
            userID VARCHAR(10) NOT NULL,
            user_tone VARCHAR(5) ,
            user_skintype VARCHAR(5) ,
            user_trouble VARCHAR(20) ,
            user_style VARCHAR(10) ,
            PRIMARY KEY(userID)
            )
            """)

for idx,row in df2.iterrows():
    try:
        sql = "INSERT INTO customer_type(userID, user_tone, user_skintype, user_trouble, user_style)\
            values(%s, %s, %s, %s, %s)"
        
        cur.execute(sql, (row['userID'],row['user_tone'],row['user_skintype'],row['user_trouble'], row['user_style']))
        
    except Exception as e:
        print("Error:", e)
        continue

conn.commit()
conn.close()

## 3.고객 화장대 DB 넣기

In [24]:
import pandas as pd 
df = pd.read_csv('./files/03.customer_regis_sampling.csv')
df

FileNotFoundError: [Errno 2] No such file or directory: './files/03.customer_regis_sampling.csv'

In [31]:

conn = pymysql.connect(host='localhost', user='root', password='apple123!!', db = 'beaulol', charset='utf8')
cur= conn.cursor()

insert_sql = "INSERT INTO customer_regis(regis_num, regis_date, userID, prd_brand, prd_name) values(%s, %s, %s, %s, %s)"
create_sql = """ 
        create table if not exists customer_regis ( 
        regis_num VARCHAR(30) NOT NULL UNIQUE,
        regis_date VARCHAR(10),
        userID VARCHAR(10) NOT NULL,
        prd_brand VARCHAR(50) NOT NULL, 
        prd_name VARCHAR(500) NOT NULL, 
        pay_date DATE, 
        pay_price VARCHAR(10),
        op_date DATE,
        PRIMARY KEY(regis_num)
        );
        """

cur.execute(create_sql)
for i in range(len(df['regis_num'])):
        cur.execute(insert_sql,(df['regis_num'][i],'2024-01-23',df['userID'][i],df['prd_brand'][i],df['prd_name'][i]))
# cur.execute(insert_sql,(df['regis_num'],'240123',df['userID'],df['prd_brand'],df['prd_name']))
        
conn.commit()
conn.close()

## 4.화장품 상품정보 DB 넣기

In [ ]:
def categorized(df,category):
    df['제품유형'] = f'{category}'
    return df

In [ ]:
import pandas as pd
cream = pd.read_csv('./output/prep_cream.csv')
lotion = pd.read_csv('./output/prep_lotion.csv')
skin = pd.read_csv('./output/prep_skin.csv')
serum = pd.read_csv('./output/prep_serum.csv')

In [ ]:
serum = categorized(serum,'세럼')
cream = categorized(cream,'크림')
skin = categorized(skin,'스킨토너')
lotion = categorized(lotion,'로션')

In [ ]:
print(serum['제품유형'].head())
print(cream['제품유형'].head())
print(skin['제품유형'].head())
print(lotion['제품유형'].head())

0    세럼
1    세럼
2    세럼
3    세럼
4    세럼
Name: 제품유형, dtype: object
0    크림
1    크림
2    크림
3    크림
4    크림
Name: 제품유형, dtype: object
0    스킨토너
1    스킨토너
2    스킨토너
3    스킨토너
4    스킨토너
Name: 제품유형, dtype: object
0    로션
1    로션
2    로션
3    로션
4    로션
Name: 제품유형, dtype: object


In [ ]:
print(serum.columns)
print(cream.columns)
print(skin.columns)
print(lotion.columns)

Index(['제품명', '브랜드', '가격', '상품정보', '썸네일', '용량', '유통기한', '성분', '제품유형'], dtype='object')
Index(['제품명', '브랜드', '가격', '상품정보', '썸네일', '용량', '유통기한', '성분', '제품유형'], dtype='object')
Index(['제품명', '브랜드', '가격', '상품정보', '썸네일', '용량', '유통기한', '성분', '제품유형'], dtype='object')
Index(['제품명', '브랜드', '가격', '상품정보', '썸네일', '용량', '유통기한', '성분', '제품유형'], dtype='object')


In [ ]:
print(serum.info())
print(cream.info())
print(skin.info())
print(lotion.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 519 entries, 0 to 518
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   제품명     519 non-null    object
 1   브랜드     519 non-null    object
 2   가격      519 non-null    object
 3   상품정보    519 non-null    object
 4   썸네일     519 non-null    object
 5   용량      519 non-null    object
 6   유통기한    519 non-null    object
 7   성분      516 non-null    object
 8   제품유형    519 non-null    object
dtypes: object(9)
memory usage: 36.6+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   제품명     48 non-null     object
 1   브랜드     48 non-null     object
 2   가격      48 non-null     object
 3   상품정보    48 non-null     object
 4   썸네일     48 non-null     object
 5   용량      48 non-null     object
 6   유통기한    48 non-null     object
 7   성분      47 non-null    

In [ ]:
import pymysql

def update_prd_info(df):
    df = df.dropna()
    df = df.drop_duplicates()
    df = df[['제품명','브랜드', '가격', '용량', '성분', '유통기한', '제품유형']]
    # Database connection
    conn = pymysql.connect(host='localhost', user='root', password='apple123!!', db='beaulol', charset='utf8')

    with conn.cursor() as curs:
        # Create table if it does not exist
        curs.execute("""
        CREATE TABLE IF NOT EXISTS product_info (
            prd_name VARCHAR(500),
            prd_brand VARCHAR(50),
            prd_price VARCHAR(10),
            prd_capa TEXT,
            prd_ingredients TEXT,
            prd_exp TEXT,
            prd_category VARCHAR(10),
            PRIMARY KEY (prd_name)
        );
        """)

        # Prepare the SQL statement for inserting data
        sql = """
        REPLACE INTO product_info (prd_name, prd_brand, prd_price, prd_capa, prd_ingredients, prd_exp, prd_category)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        """

        # Convert DataFrame to a list of tuples
        data_tuples = [tuple(x) for x in df.to_numpy()]

        # Execute the SQL statement in bulk
        curs.executemany(sql, data_tuples)

    # Commit changes and close connection
    conn.commit()
    conn.close()


In [ ]:
update_prd_info(cream)
update_prd_info(lotion)
update_prd_info(serum)
update_prd_info(skin)

## 5.리뷰 DB 넣기

In [28]:
import pandas as pd
csv_file ='./files/05.oliveyoung_raw_review.csv'
df = pd.read_csv(csv_file)
df

,brand,prod_name,id,star_rate,review,keyword
0,브이티,VT 리들샷 100 에센스,간호산데여,5,환절기에 피부 뒤집어져서 반신반의 하며 샀는데 지인짜 효과 지대로에요ㅎㅎㅎㅎ바를때 ...,효과가 좋아요/진정이 돼요.
1,브이티,VT 리들샷 100 에센스,올영메이트인데요,5,온라인으로 주문하고 한달동안 사용해봤는데 이번주부터 우리 매장에도 들어왔다~~~자연...,피부결이 개선됐어요.
2,브이티,VT 리들샷 100 에센스,hareen,5,아주 잘쓰고 있어요!!!! 처음에 따가운게 적응이 안됐는데자주쓰니까 적응이 되네여..!!,-
3,브이티,VT 리들샷 100 에센스,콩이파더22,5,리들샷 요즘 엄청 핫 하죠다이소에서 파는 것도 겨우 구해서 쓰다가 올영에서 본품 구...,"효과가 좋아요. ,진정이 됩니다."
4,브이티,VT 리들샷 100 에센스,이단콩,5,리들샷 특성상 시카 니들이 포함돼있어 자극이 느껴지는건 당연한 부분이예요〰️다만 리...,트러블이 줄었어요/붉은기가 나아졌어요/모공이 줄었어요.
...,...,...,...,...,...,...
995,브이티,VT 리들샷 100 에센스,도제,3,려드름 피부도 쓸수있다 했는디저는 딱히 효과가 없는듯해요ㅠㅍ,NaN
996,브이티,VT 리들샷 100 에센스,당토,2,자극은 그렇다치고... 이거 사용하니까 조그마한 좁쌀모양 피지가 얼굴 전체에 생겼습...,NaN
997,브이티,VT 리들샷 100 에센스,어도러블여왕님,4,계속 쓰다보니 점점 안 따가워지는 것 같아요 다음엔 더 센 걸로 사보려구요!,NaN
998,브이티,VT 리들샷 100 에센스,써보고좋으면짖음,5,아무래도 자극이 느껴지지만 자연스러운 현상이라고 하니 꾸준히 써볼 생각입니다,NaN


In [29]:
conn = pymysql.connect(host='localhost', user='root', password='apple123!!',\
    db = 'beaulol', charset='utf8')
cur= conn.cursor()

f=open('./files/05.oliveyoung_raw_review.csv','r')

csvReader = list(csv.reader(f))
cur.execute("create table if not exists review_db (prd_brand VARCHAR(20), prd_name VARCHAR(40), userID VARCHAR(20),\
star_rate INT, review TEXT, PRIMARY KEY (userID))")


for data in csvReader[1:]:
    row1= data[0]
    row2= data[1]
    row3= data[2]
    row4= int(data[3])
    row5= data[4]
    
    print(row3,row4,type(row4))
    
    try:
        sql = """insert into review_db (prd_brand, prd_name, userID, star_rate, review) values(%s, %s, %s, %s, %s);"""
        cur.execute(sql,(row1,row2,row3,row4,row5))
    except:
        continue
    
f.close()
conn.commit()


conn.close()

간호산데여 5 <class 'int'>
올영메이트인데요 5 <class 'int'>
hareen 5 <class 'int'>
콩이파더22 5 <class 'int'>
이단콩 5 <class 'int'>
올올덕 5 <class 'int'>
안녕하숙까 5 <class 'int'>
jwwcat 5 <class 'int'>
비니수 5 <class 'int'>
Mizthecat 5 <class 'int'>
syjal 5 <class 'int'>
bear베어 5 <class 'int'>
냐얀야나냥 5 <class 'int'>
야야아야양 5 <class 'int'>
푸링짱 5 <class 'int'>
jio11 5 <class 'int'>
jio11 5 <class 'int'>
오롤롤 5 <class 'int'>
hq1 5 <class 'int'>
체리쿵 5 <class 'int'>
부타로니 5 <class 'int'>
코투닝 5 <class 'int'>
미미미mme 5 <class 'int'>
코코아녕 3 <class 'int'>
Likey 5 <class 'int'>
마리안네 4 <class 'int'>
경의선귀여운죄안된다 5 <class 'int'>
자신있써어어 5 <class 'int'>
푸링짱 5 <class 'int'>
김먕먕 5 <class 'int'>
메리크 5 <class 'int'>
O3OY 5 <class 'int'>
Channri 5 <class 'int'>
볼빵빵춘식이 5 <class 'int'>
bear베어 5 <class 'int'>
whvely 5 <class 'int'>
sol솔 3 <class 'int'>
충동구매니니즈 4 <class 'int'>
정연니 1 <class 'int'>
살랑이 4 <class 'int'>
Myzu 5 <class 'int'>
콩이맘맘 5 <class 'int'>
올영에진심인편 5 <class 'int'>
sol솔 3 <class 'int'>
올챙임 4 <class 'int'>
하늘을나는고래 5 <class 'i

## 6.제거대상용매 DB 넣기

In [1]:
import pandas as pd
solvent = pd.read_csv('./files/06.solvent_EK.csv')
solvent

,INGR_ENG_NAME,INGR_KOR_NAME
0,CYCLOTETRASILOXANE,사이클로테트라실록세인
1,TETRADECENE,테트라데센
2,METHYL TRIMETHICONE,메틸트라이메티콘
3,OCTADECANE,옥타데케인
4,PEG-18,피이지-18
...,...,...
595,CETEARTRIMONIUM CHLORIDE,세테아트라이모늄클로라이드
596,EGG SHELL MEMBRANE EXTRACT,난각막추출물
597,O-CYMEN-5-OL,o-사이멘-5-올
598,POTASSIUM METABISULFITE,포타슘메타바이설파이트


In [2]:
import pymysql
import csv
import pandas as pd
solvent = pd.read_csv('./files/06.solvent_EK.csv')
conn = pymysql.connect(host='localhost', user='root', password='apple123!!', db = 'beaulol', charset='utf8')
cur = conn.cursor()

cur.execute("CREATE TABLE IF NOT EXISTS solvent (ingredient_eng VARCHAR(255), ingredient_kor VARCHAR(255))")

for i in range(len(solvent)):
    eng_name = solvent['INGR_ENG_NAME'][i]
    kor_name = solvent['INGR_KOR_NAME'][i]
    sql = "INSERT INTO solvent (ingredient_eng, ingredient_kor) VALUES (%s, %s);"
    cur.execute(sql, (eng_name, kor_name))

conn.commit()
conn.close()